<img align="left" src = https://project.lsst.org/sites/default/files/Rubin-O-Logo_0.png width=250 style="padding: 10px"> 
<b>Introduction to Source Detection</b> <br>
Contact authors: Douglas Tucker, Alex Drlica-Wagner<br>
Last verified to run: 2022-09-29 <br>
LSST Science Piplines version: Weekly 2022_40 <br>
Container Size: medium <br>
Targeted learning level: Intermediate <br>

**Description:** Access, display, and manipulate images; detect, deblend, and measure sources; and extract, plot, and use object footprints.

**Skills:** Run source detection, deblending and measurement. Use source "footprints".

**LSST Data Products:** Butler calexp images and mask planes.

**Packages:** lsst.pipe.tasks.characterizeImage, lsst.meas.algorithms.detection, lsst.meas.deblender, lsst.meas.base

**Credit:**
Originally developed by Alex Drlica-Wagner and Imran Hasan in the context of the LSST Stack Club.

**Get Support:**
Find DP0-related documentation and resources at <a href="https://dp0-2.lsst.io">dp0-2.lsst.io</a>. Questions are welcome as new topics in the <a href="https://community.lsst.org/c/support/dp0">Support - Data Preview 0 Category</a> of the Rubin Community Forum. Rubin staff will respond to all questions posted there.

## 1. Introduction

This notebook provides a brief introduction to running the LSST Science Pipelines source detection, measurement, and deblending algorithms. It does not go into depth about optimizing detection, measurement, or deblending parameters for different types of sources. 

Some source detection and measurement details come from Robert Lupton's [Tune Detection.ipynb](https://github.com/RobertLuptonTheGood/notebooks/blob/master/Demos/Tune%20Detection.ipynb) and [Kron.ipynb](https://github.com/RobertLuptonTheGood/notebooks/blob/master/Demos/Kron.ipynb).
Interaction with `lsst.afw.display` was also improved by studying Michael Wood-Vasey's [DC2_Postage Stamps.ipynb](https://github.com/LSSTDESC/DC2-analysis/blob/master/tutorials/dm_butler_postage_stamps.ipynb).
More information on footprints can be found on the Stack Club notebook by Imran Hasan [here](https://github.com/LSSTScienceCollaborations/StackClub/blob/master/SourceDetection/Footprints.ipynb).

After working through this notebook you should be able to run the `lsst.meas.algorithm` source detection, deblending, and measurement tasks; plot the resulting source catalogs on an image; and examine the `footprint` of the detected sources.  Other techniques that are demonstrated (but not emphasized) include the use of the `butler` to access a specific `calexp`, the creation of an image cutout; and the and use of `lsst.afw.display` to plot the image cutout.

### 1.1. Package Imports

The [`matplotlib`](https://matplotlib.org/) library is a widely used Python plot library. 

We load the `lsst.afw.display` library to gain access to the image visualization routines we'd like to use, and the `lsst.daf.butler` library, which is used to access data products.  The `lsst.afw.image` and `lsst.afw.table` libraries provide handlers, respectively for the image and table outputs of the LSST Science Pipelines.  The `lsst.afw.geom` provide sky- and LSST-specific geometric routines. 

As we will see later, the `lsst.pipe.tasks.characterizeImage` library is used for characterizing properties of a given image (like PSF), the `lsst.meas.algorithms.detection` library provides methods for detecting sources within an image, the `lsst.meas.deblender` library gives methods for deblending sources into component parts ("children"), and the `lsst.meas.base` includes methods for measuring the properties of sources.

In [ ]:
# General python packages
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable

# LSST Science Pipeline packages (see pipelines.lsst.io)
import lsst.daf.base as dafBase
from lsst.daf.butler import Butler
import lsst.afw.image as afwImage
import lsst.afw.display as afwDisplay
import lsst.afw.table as afwTable
import lsst.geom as geom

# Pipeline tasks
from lsst.pipe.tasks.characterizeImage import CharacterizeImageTask
from lsst.meas.algorithms.detection import SourceDetectionTask
from lsst.meas.deblender import SourceDeblendTask
from lsst.meas.base import SingleFrameMeasurementTask

### 1.2. Define functions and parameters

Set the matplotlib plot color table.

In [ ]:
plt.style.use('tableau-colorblind10')

Let us also set `lsst.afw.display` to use the `matplotlib` backend:

In [ ]:
afwDisplay.setDefaultBackend('matplotlib')
plt.rcParams['figure.figsize'] = (8.0, 8.0)

## 2. Data Access


### 2.1. Access the `calexp` Data

Here we use the `butler` to access a `calexp` from the DP0.2 dataset. More information on the `butler` and `calexp`, 
and how to determine the `dataId` (e.g., the visit and detector numbers) are available in other tutorials. 
We assume that this information is known.

First, let us define the configuration and collection and instantiate an instance of the butler.

In [ ]:
# For DP0.2, we want the following configuration and collection:
config = "dp02"
collection = '2.2i/runs/DP0.2'

butler = Butler(config=config, collections=collection)

Let us look at the `calexp` data for `detector` 75 in `visit` 512055.

In [ ]:
# Define the dataId using just visit and detector
dataId = {'visit': 512055, 'detector': 75}

# Use the butler to get the calexp
calexp = butler.get('calexp', **dataId)

<br>

As described in other tutorials, the `calexp` object possess more than just the raw pixel data of the image. It also contains a `mask`, which stores information about various pixels in a bit mask.

Here are some optional commands to explore the calexp. Uncomment one of the code lines to learn more.

In [ ]:
# If you want to investigate the contents of the masked image:
# calexp.maskedImage

# If you just want one of the three components:
# calexp.maskedImage.image
# calexp.maskedImage.mask
# calexp.maskedImage.variance

# These also work:
# calexp.image
# calexp.mask
# calexp.variance

# The calexp also contains the PSF, the WCS, and the photometric calibration
# calexp.getPsf()
# calexp.getWcs()
# calexp.getPhotoCalib()

Since we are interested in performing our own source detection and measurement, we choose to clear the existing `DETECTED` mask plane.

In [ ]:
# Unset the `DETECTED` bits of the mask plane
calexp.mask.removeAndClearMaskPlane('DETECTED')

In [ ]:
# Plot the calexp we just retrieved
plt.figure()
afw_display = afwDisplay.Display()
afw_display.scale('asinh', 'zscale')
afw_display.mtv(calexp.image)

<br>

### 2.2. Add the Subtracted Sky Background Back into the Image

Here we retrieve the subtracted background for the same dataId and add it back into the image. This section is optional.

First, we obtain the `calexpBackground` object for this `dataId`.  We will again use the `butler`.

In [ ]:
bkgd = butler.get('calexpBackground', **dataId)

Now, let us display the background we obtained.

In [ ]:
plt.figure()
afw_display = afwDisplay.Display()
afw_display.scale('linear', 'zscale')
afw_display.mtv(bkgd.getImage())
plt.title("Local Polynomial Background")

Next, we add the background into the `calexp`, and re-display the `calexp`. Note the scale in the sidebar now goes up to thousands of counts instead of hundreds of counts.

In [ ]:
# Note: executing this cell multiple times will add the background
#  multiple times
calexp.maskedImage += bkgd.getImage()

In [ ]:
plt.figure()
afw_display = afwDisplay.Display()
afw_display.scale('asinh', 'zscale')
afw_display.mtv(calexp.image)

## 3. Source Detection, Deblending, and Measurement

We now want to run the LSST Science Pipelines' source detection, deblending, and measurement tasks. While we run all three tasks, this notebook is mostly focused on the detection of sources.

Recall that these tasks were imported up at the top of this notebook, from `lsst.pipe` and `lsst.meas`. More information can be found at [pipelines.lsst.io](https://pipelines.lsst.io/) (the search bar at the top left of that page is a very handy way to find documentation for a specific task).

We start by creating a minimal schema for the source table. The schema describes the output properties that will be measured for each source. This schema will be passed to all of the tasks, as we call each in turn, and each task will add columns to this schema as it measures sources in the image.

In [ ]:
# Create a basic schema to use with these tasks
schema = afwTable.SourceTable.makeMinimalSchema()
print(schema)

# Create a container which will be used to record metadata
#  about algorithm execution
algMetadata = dafBase.PropertyList()
print('algMetadata: ')
algMetadata

### 3.1. Configuring Tasks

Each task possesses an associated configuration class. The properties of these configuration classes can be determined from the classes themselves.

In [ ]:
# Uncomment the following line to view help
#  for the CharacterizeImageTask configuration
# Replace 'CharacterizeImageTask' with a different
#  task name to view additional help information

# help(CharacterizeImageTask.ConfigClass())

As a starting point, like the `schema` and `algMetadata` above, here we set some basic config parameters and instantiate the tasks to get you started. In this case, we configure several different tasks:

* CharacterizeImageTask: Characterizes the image properties (e.g., PSF, etc.)
* SourceDetectionTask: Detects sources
* SourceDeblendTask: Deblend sources into constituent "children"
* SingleFrameMeasurementTask: Measures source properties

In [ ]:
# Characterize the image properties
config = CharacterizeImageTask.ConfigClass()
config.psfIterations = 1
charImageTask = CharacterizeImageTask(config=config)

# Detect sources
config = SourceDetectionTask.ConfigClass()
# detection threshold in units of thresholdType
config.thresholdValue = 10
# units for thresholdValue
config.thresholdType = "stdev"
sourceDetectionTask = SourceDetectionTask(schema=schema, config=config)

# Deblend sources
sourceDeblendTask = SourceDeblendTask(schema=schema)

# Measure source properties
config = SingleFrameMeasurementTask.ConfigClass()
sourceMeasurementTask = SingleFrameMeasurementTask(schema=schema,
                                                   config=config,
                                                   algMetadata=algMetadata)

Note that if you want to change the value of a config parameter (e.g., `psfIterations`), do not change it in the already constructed task. Instead, change your config object and then construct a new characterize image task. Like so:
> `config.psfIterations = 3` <br>
> `charImageTask = CharacterizeImageTask(config=config)`

Like the configs, we can use `help` to explore each task and the methods used to run it.

In [ ]:
# help(charImageTask)

# Uncomment the following line, position your cursor after the period,
#  and press tab to see a list of all methods. Then recomment the line
#  because "Task." is not executable and will cause an error.
# charImageTask.

# Use the help function on any of the methods to learn more:
# help(charImageTask.writeSchemas)

# E.g., find out what options there are for config.thresholdType
# help(SourceDetectionTask.ConfigClass)

With the each of the tasks configured, we can now move on to running the source detection, deblending, and measurement. First we create `SourceTable` for holding the output of our source analysis. The columns and characteristics of this table are defined by the `schema` that we created in our configuration step.

In [ ]:
tab = afwTable.SourceTable.make(schema)

### 3.2. Image Characterization

Next we characterize our image. This calculates various global properties of the image, such as the full-width half-max of its point spread function (PSF FWHM).

In [ ]:
# Image characterization (this cell may take a few seconds)
result = charImageTask.run(calexp)

# Define the pixel coordinates of a point of interest
# (in this case, basically a random point within the image)
x_target, y_target = 1700, 2100
width, height = 400, 400
xmin, ymin = x_target-width//2, y_target-height//2
point = geom.Point2D(x_target, y_target)

# Get the PSF at our point of interest
psf = calexp.getPsf()
sigma = psf.computeShape(point).getDeterminantRadius()
pixelScale = calexp.getWcs().getPixelScale().asArcseconds()

# The factor of 2.355 converts from std to fwhm
print('psf fwhm = {:.2f} arcsec'.format(sigma*pixelScale*2.355))

With the image characterized, we are now interested in running the source detection, deblending, and measurement tasks. Each of these tasks is called with the `run` method. The parameters of this method can be investigated using `help`.

In [ ]:
# We are specifically interested in the `SourceMeasurementTask`
# help(sourceMeasurementTask.run)

In [ ]:
# Source detection (this cell may take a few seconds)
result = sourceDetectionTask.run(tab, calexp)
type(result)

The source detection task returns an [`lsst.pipe.base.struct.Struct`](http://doxygen.lsst.codes/stack/doxygen/x_masterDoxyDoc/classlsst_1_1pipe_1_1base_1_1struct_1_1_struct.html). A `Struct` is just a generalized container for storing multiple output components and accessessing them as attributes. The content of this `Struct` can be investigated with the `getDict` method.

In [ ]:
for k, v in result.getDict().items():
    print(k, type(v))

In [ ]:
result.numPosPeaks

When we use `config.thresholdValue = 10` we get about 940 positive peaks. If we rerun with `config.thresholdValue = 3` we would find about 5340 positive peaks.  (Exact numbers for this image depend on version of the DM Stack used and version the configuration files employed.)

In [ ]:
sources = result.sources

Most of the columns for sources will be NaN at first.

In [ ]:
# Uncomment to see that most values are NaN
# sources

Note that if we desire we can save these processed sources to disk as FITS tables:

In [ ]:
# sources.writeFits("outputTable.fits")
# calexp.writeFits("example1-out.fits")

### 3.3. Deblend and Measure Sources

Next we run the `SourceDeblendTask` and `SingleFrameMeasurementTask`. A deeper investigation of these tasks is beyond the scope of this notebook.

In [ ]:
# Source deblending
sourceDeblendTask.run(calexp, sources)

# Source measurement
sourceMeasurementTask.run(measCat=sources, exposure=calexp)

To get a better look at the output sources, we need to make sure that the `SourceCatalog` is contiguous in memory. Converting to an `astropy` table provides a human-readable output format. A deeper dive into `SourceCatalog` is beyond the scope of this notebook.

In [ ]:
# The copy makes sure that the sources are sequential in memory
sources = sources.copy(True)

# Investigate the output source catalog
sources.asAstropy()

### 3.4. Overplot Sources on Image

We can now overplot our detected sources on the calexp or a cutout image using `afwDisplay`.

Let's do this for a cutout image instead of the full calexp.  To generate a cutout image of the calexp, create a bounding box and passing it to the `Factory` method of our calexp (a `lsst.afw.image.Exposure` object). Below we explain the specific arguments that we are passing to `Factory`:
```
calexp : the ExposureF we are starting from
bbox   : the bounding box of the cutout
origin : the image pixel origin is local to the cutout array
deep   : copy the data rather than passing by reference
```

<a id='display-error'></a>

In [ ]:
# Define a small region for a cutout
bbox = geom.Box2I()
bbox.include(geom.Point2I(xmin, ymin))
bbox.include(geom.Point2I(xmin + width, ymin + height))

# An alternative way to defined the same cutout region
# bbox = geom.Box2I(geom.Point2I(xmin, ymin), geom.Extent2I(width, height))

# Generate the cutout image
cutout = calexp.Factory(calexp, bbox, origin=afwImage.LOCAL, deep=False)

In [ ]:
# Display the cutout and sources with afw display
image = cutout.image
plt.figure()
afw_display = afwDisplay.Display()
afw_display.scale('asinh', 'zscale')
afw_display.mtv(image)
plt.gca().axis('off')

# We use display buffering to avoid re-drawing the image
#  after each source is plotted
with afw_display.Buffering():
    for s in sources:
        afw_display.dot('+', s.getX(), s.getY(), ctype=afwDisplay.RED)
        afw_display.dot('o', s.getX(), s.getY(), size=20, ctype='orange')

## 4. Footprints

Object footprints are an integral component of the high-level CCD processing tasks (e.g., detection, measurement, and deblending). To quote [Bosch et al. (2017)](https://arxiv.org/pdf/1705.06766.pdf), 

> Footprints record the exact above-threshold detection region on a CCD. These are similar to  SExtractor’s “segmentation map", in that they identify which pixels belong to which detected objects

This quote draws an analogy between footprints and segmentation maps, since they both identify pixels with values above some threshold. This is a useful similarity, since it gives us a place to start understanding the properties of footprints.

The result of the `SourceDetectionTask` stores the footprints associated to detected objects

In [ ]:
# Grab the above-threshold footprints that were detected,
#  and assign them to the variable `fps`
fpset = result.positive
fps = fpset.getFootprints()

In [ ]:
# We can get a rough view of the first source's footprint from its span
fps[0].getSpans()

You can almost see the footprint by looking at the 1's and 0's here. Keep in mind that the first row of this array will be the *bottom* row of the image. Later, when we display the footprint, its general pattern will appear "upside down" compared to this pattern of 1s. 

### 4.1. Heavy Footprints

At the moment, our footprints indicate which pixels they consist of, but not the values of those pixels from the image. To extract the actual values of the pixels that correspond to the ones in the footprint span, we need to convert our footprint into a `HeavyFootprint`. HeavyFootprints have all of the qualities of Footprints, but additionally 'know' about pixel level data from the image, variance, and mask planes.

In [ ]:
# First we demonstrate that the footprint is NOT heavy
fps[0].isHeavy()

In [ ]:
# Next, we make all the footprints heavy at the same time
#  by operating on the footprint set
fpset.makeHeavy(calexp.getMaskedImage())

# This means we have to redefine fps:
hfps = fpset.getFootprints()

In [ ]:
# All of the arrays here will be flattend 1D arrays
#  of pixels from the footprint.
# Uncomment this line to print the array and see that
#  now, it contains pixel values.

# hfps[0].getImageArray()

Now we can use the span set to reassemble the image array into the footprint. Above we saw that the image array is a 1D numpy array, but the footprint itself is 2 dimensional. Fortunately, the span set has an `unflatten` method that rearranges the image array into the proper 2 dimensional shape. If you want to change the colormap, see [matplotlib colormap options](https://matplotlib.org/stable/tutorials/colors/colormaps.html).

In [ ]:
plt.figure()
plt.imshow(fps[0].getSpans().unflatten(hfps[0].getImageArray()),
           cmap='bone', origin='lower')

The heavy footprint also comes with a 1d mask array.

In [ ]:
hfps[0].getMaskArray()

In [ ]:
calexp.getMask().getMaskPlaneDict()

The values are the exponent of the bitmask. So pixels only marked detected will be 2^5 = 32. Pixels that are both on the edge of the original image and detected will be 2^5 + 2^4 = 48. We will visualize the mask plane values in a similar manner as before, except that we will be displaying the values of the mask array.

In [ ]:
plt.figure()
im = plt.imshow(fps[0].getSpans().unflatten(hfps[0].getMaskArray()),
                origin='lower')

# Create a new axis, "cax" on the right side of the image display.
# The width of cax will be 5% of the axis "ax".
# The padding between cax and ax will be 1% of the axis.
ax = plt.gca()
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad="1%")
plt.colorbar(im, cax=cax, ticks=[0, 32, 32+16])